Tabris Loveless
11/7/22

In [12]:
import numpy as np
import requests
from scipy.optimize import minimize
from numba import jit

def get_pos_from_url(address='http://doye.chem.ox.ac.uk/jon/structures/LJ/points/', N=7):
    url_address = address + str(N)
    data_str = requests.get(url_address).text
    return parse_url_text(data_str)    
    
def parse_url_text(data_str):
    x_array = []
    text = data_str.split('\n')
    for line in text:
        [x_array.append(float(i)) for i in line.split()]
    return np.array(x_array)

@jit
def lj(r):   
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

#calculate every position
@jit
def total_energy(positions):
    """
    Calculates the total energy of a given system
    """
    E = 0
    n_atom = int(len(positions)/3)
    
    #assuming [x0 y0, z0, x1, y1, z1]
    for i in range(n_atom-1):
        for j in range(i+1, n_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += lj(dist)
    return E

@jit
def init_pos(N, L = 5):
    return L*np.random.random_sample((N*3,))

In [7]:
pos = init_pos(10)
res = minimize(total_energy, pos, method='CG', tol=1e-4)
pos2 = get_pos_from_url(N=20)
print('Website Min: ',total_energy(pos2))
print('Calculated Min: ', res.fun) 

Website Min:  -77.17704248805858
Calculated Min:  -26.62331977141552


In [9]:
f_values = []
x_values = []
N_attempts = 2
N_atom = 10
for i in range(N_attempts):
    pos = init_pos(N_atom)
    res = minimize(total_energy, pos, method='CG', tol=1e-4)
    f_values.append(res.fun)
    x_values.append(res.x)
    if i%10==0:
        print('step: ', i, '  values:', res.fun)

print('The global minimum:  ', min(f_values))

step:  0   values: -18.778208174755772
The global minimum:   -27.11490952739437


In [14]:
def singlemin(atoms, N):
    energy = []
    for i in range(N):
        pos = init_pos(atoms)
        res = minimize(total_energy, pos, method='CG', tol=1e-4)
        energy.append(res.fun)

    pos2 = get_pos_from_url(N=atoms)
    webmin = total_energy(pos2)
    print('Website Min: ',total_energy(pos2))
    print(energy)
    print('Calculated Min: ', min(energy))
    
singlemin(10, 10)

Website Min:  -28.422531893437565
[-26.418356106786387, -26.52174160395751, -28.422531893388545, -26.44288499191578, -25.50400788959803, -26.442884991291688, -25.40748183944518, -25.288749212521765, -26.62331977166693, -27.545206852119104]
Calculated Min:  -28.422531893388545
